In [4]:
import pandas as pd
from pathlib import Path
import json
import pendulum
import numpy as np

In [6]:
in_dir = "/Users/kay/Papers/论文合作/于孟利/weibo_data_analysis-20210210/weibo_data_json"

In [88]:
def day_night(time_list):
    '''
    发微博的时间序列转化为4个时间段的统计
    :param in_name:
    :param out_name:
    :return:
    '''
    cnt = [0] * 24
    for t in time_list:
        hour = t.hour
        # print(hour)
        # 分为四个时间段
        # cnt[int(int(hour) / 6)] += 1
        # 分为24小时
        cnt[int(hour)] += 1
    return cnt


def weeks(time_list):
    '''
    发微博的时间序列转化为7个时间段的统计
    :param in_name:
    :param out_name:
    :return:
    '''
    cnt = [0] * 7
    for t in time_list:
        weekday = t.weekday()
        cnt[int(weekday)] += 1
    return cnt


def time_interval(time_list):
    interval = []
    for i in np.arange(1, len(time_list)):
        t2 = time_list[i-1]
        t1 = time_list[i]
        _inter = (t2 - t1).total_seconds()
        if _inter < 86400 * 7:
            interval.append(_inter / 3600)
    # return interval
    interval = np.array(interval)
    # print(interval)
    return interval.mean(), interval.std()


def life_length(time_list):
    return int((max(time_list) - min(time_list)).total_seconds() / 3600 / 24) + 1

In [33]:
# 时间分析

'''
Basic features. Basic features are selected to reﬂect the user’s demographics, preliminary statuses and elementary interactions on social media, including gender, tweeting patterns and privacy settings. Speciﬁcally, tweeting patterns contain log(AUW + 1) (where AUW is the age of a user on Weibo in units of days), log(NT + 1) (where NT is the total number of tweets the user posted), log(NT/(AUW + 1)) (the frequency of posting), log(NFER + 1) (where NFER is deﬁned as the number of the user’s followers), log(NFEE + 1) (where NFEE denotes the number of the user’s followees), NT/(NFER + 1), and NT/(NFEE + 1). With respect to the privacy settings, corresponding binary features indicate whether a user allows comments from others, whether the user allows private messages sent from others and whether the user allows Weibo to track their real-time location. In addition, we consider the length of self-description as a feature.
'''
users_features = []

for in_name in Path(in_dir).rglob("*.json"):
    # print(in_name)
    d = json.load(open(in_name))
    user = d["user"]
    u = {
        "userID_num": user["id"],
        "NT": np.log(user["weibo_num"] + 1),
        "NFEE": np.log(user["following"] + 1),
        "NEER": np.log(user["followers"] + 1),
        "NT / NFEE": (user["weibo_num"] + 1) / (user["following"] + 1),
        "NT / NFER": (user["weibo_num"] + 1) / (user["followers"] + 1),
        "NEEE / NFER": (user["following"] + 1) / (user["followers"] + 1),
    }
    users_features.append(u)
    # print(user)

len(users_features)

df = pd.DataFrame(users_features).set_index("userID_num")
df.to_csv("data/MengLi_users_Basic_features.csv", float_format="%.4f")
df


,NT,NFEE,NEER,NT / NFEE,NT / NFER,NEEE / NFER
userID_num,,,,,,
2982254865,6.063785,5.093750,5.407172,2.638037,1.928251,0.730942
3943065337,9.169727,6.356108,7.021084,16.670139,8.573214,0.514286
3968712237,5.743003,5.860786,5.176150,0.888889,1.762712,1.983051
5013416876,6.495266,5.852202,5.402677,1.902299,2.981982,1.567568
5230736039,4.976734,5.347108,4.820282,0.690476,1.169355,1.693548
...,...,...,...,...,...,...
1278570913,5.897154,5.293305,4.094345,1.829146,6.066667,3.316667
5236316239,7.216709,5.517453,4.189655,5.469880,20.636364,3.772727
2678428523,6.075346,4.304065,5.676754,5.878378,1.489726,0.253425


In [ ]:
df = pd.DataFrame(users_features).set_index("uid")
df.to_csv("data/MengLi_users_Time_features.csv", float_format="%.4f")

In [28]:
survey_data = pd.read_excel("/Users/kay/Papers/论文合作/于孟利/weibo_data_analysis-20210210/456_ID_data.xlsx")

In [29]:
survey_data = survey_data.set_index("userID_num")

In [17]:
survey_data.join(df)

NameError: name 'df' is not defined

In [30]:
basic_features = pd.read_csv("data/MengLi_users_Bas_features.csv")
basic_features

,userID_num,NT,NFEE,NEER,NEEE / NFER
0,2982254865,6.0638,5.0938,5.4072,-0.3134
1,3943065337,9.1697,6.3561,7.0211,-0.6650
2,3968712237,5.7430,5.8608,5.1761,0.6846
3,5013416876,6.4953,5.8522,5.4027,0.4495
4,5230736039,4.9767,5.3471,4.8203,0.5268
...,...,...,...,...,...
454,1278570913,5.8972,5.2933,4.0943,1.1990
455,5236316239,7.2167,5.5175,4.1897,1.3278
456,2678428523,6.0753,4.3041,5.6768,-1.3727
457,3078158207,5.6699,5.5530,5.4681,0.0849


In [31]:
survey_data.join(basic_features)

,num,提交答卷时间,所用时间,nickname,userURL,userID,sex,age,Born,专业,...,高兴,低落,恐惧,是否有情绪,微博数量,userID_num,NT,NFEE,NEER,NEEE / NFER
userID_num,,,,,,,,,,,,,,,,,,,,,
5822697591,10,2019/10/20 16:34:17,856秒,满天小xingx,https://weibo.com/u/5822697591,u/5822697591,2,23,1996,4,...,0.749,0.097,0.018,1,549,NaN,NaN,NaN,NaN,NaN
5866763968,18,2019/10/20 16:42:25,883秒,不要放蘑菇,https://weibo.com/u/5866763968,u/5866763968,1,20,1999,4,...,0.583,0.000,0.250,1,12,NaN,NaN,NaN,NaN,NaN
5311953749,28,2019/10/20 16:49:03,1440秒,彭老师不爱留作业,https://weibo.com/u/5311953749,u/5311953749,1,24,1995,4,...,0.799,0.058,0.018,1,622,NaN,NaN,NaN,NaN,NaN
2366180120,31,2019/10/20 16:51:01,1024秒,cryptobiote,https://weibo.com/u/2366180120,u/2366180120,1,25,1994,1,...,0.770,0.053,0.026,1,152,NaN,NaN,NaN,NaN,NaN
2769435005,32,2019/10/20 16:51:13,1090秒,不是很紧,https://weibo.com/u/2769435005,u/2769435005,1,22,1997,1,...,0.329,0.268,0.122,1,82,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139304270,1450,2019/11/15 11:32:33,929秒,_怪兽回收处处长,https://weibo.com/u/2139304270,u/2139304270,1,21,1998,2,...,0.643,0.107,0.000,1,28,NaN,NaN,NaN,NaN,NaN
1822574395,1454,2019/11/15 11:48:23,1235秒,夷歆雯,https://weibo.com/u/1822574395,u/1822574395,2,21,1998,3,...,0.638,0.188,0.025,1,1261,NaN,NaN,NaN,NaN,NaN
6249948493,1461,2019/11/16 14:54:01,1002秒,·松栗奶油,https://weibo.com/u/6249948493,u/6249948493,2,20,1999,4,...,0.636,0.091,0.000,1,11,NaN,NaN,NaN,NaN,NaN
